In [1]:
print("Kernel Working Fine!")

Kernel Working Fine!


## Loading the data

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
loader_HP = PyPDFLoader("Data/harrypotter.pdf")

In [4]:
doc_HP = loader_HP.load()

In [5]:
print(len(doc_HP))

3623


In [6]:
loader_got = PyPDFLoader("Data/got.pdf")

In [7]:
doc_got = loader_got.load()

In [8]:
print(len(doc_got))

755


## Text Splitting / Chunking

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [11]:
text_HP = text_splitter.split_documents(doc_HP)

In [13]:
text_got = text_splitter.split_documents(doc_got)

In [16]:
print(len(text_HP))

16976


In [17]:
print(len(text_got))

4434


## Loading the data Embedding Model

The embedding model is used to convert the chunks which we just created into vectors and then we will store these vector embeddings in a vector database.

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings, HuggingFaceBgeEmbeddings

In [19]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_88106/3585289676.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/atharvabot7/Downloads/Reranking/reranking/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
hf_bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en")

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_88106/2125299706.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en")
